<a href="https://colab.research.google.com/github/ajmainrahman/Supervised-Machine-Learning-ProblemTypes-Tasks-UseCases/blob/main/SpamvsNon_spamdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import kagglehub
path = kagglehub.dataset_download("brahimettanany/email-classification-detecting-spam-vs-non-spam")

Using Colab cache for faster access to the 'email-classification-detecting-spam-vs-non-spam' dataset.


In [3]:
path

'/kaggle/input/email-classification-detecting-spam-vs-non-spam'

In [4]:
import os

# List the contents of the directory
print(os.listdir(path))

['mail_data.csv']


In [5]:
df = pd.read_csv(os.path.join(path, 'mail_data.csv'))

print(f"First 10 rows of the data:\n")
display(df.head(10))

First 10 rows of the data:



,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [6]:
print(df['Category'].value_counts())

Category
ham     4825
spam     747
Name: count, dtype: int64


In [7]:
df.isnull().sum()

,0
Category,0
Message,0


In [8]:
!pwd

/content


In [9]:
!ls

sample_data


In [10]:
!git clone https://github.com/ajmainrahman/Supervised-Machine-Learning-ProblemTypes-Tasks-UseCases

Cloning into 'Supervised-Machine-Learning-ProblemTypes-Tasks-UseCases'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 85 (delta 36), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 30.95 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [11]:
%cd Supervised-Machine-Learning-ProblemTypes-Tasks-UseCases/Classification/Binary-Classification

/content/Supervised-Machine-Learning-ProblemTypes-Tasks-UseCases/Classification/Binary-Classification


In [12]:
!ls

README.md


In [13]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [14]:
2+3-5

0